Standard imports.


In [9]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

Propagating distribution and its derivative. They serve as reference and as initial condition. 

In [10]:
def propagatingFunctionStationary(x, xmin, xmax):
    return np.cos((2. * np.pi / ((2. / 3.) * (xmax - xmin))) * x)

def propagatingFunctionStationaryDerivative(x, xmin, xmax):
  return -(2. * np.pi / ((2. / 3.) * (xmax - xmin))) * np.sin((2. * np.pi / ((2. / 3.) * (xmax - xmin))) * x)

Solution function. Rows span time, columns span space. We have ```M + 1``` rows: ```M``` iterations + ```1``` initial condition. We have ```N + 1``` columns: ```N``` discretization points + ```1``` boundary condition.

Leapfrog.

In [11]:
def leapfrogStationary(x, t, v, xmin, xmax, alpha):

  u     = np.zeros(((M + 1), N + 1))

  # --- First step
  u[0, 0 : N + 1] = 0.5 * (propagatingFunctionStationary(x - xmin - v * t[0], xmin, xmax) - propagatingFunctionStationary(x - xmin + v * t[0], xmin, xmax))       
  
  # --- Second step
  # --- Left boundary condition
  u[1, 0]         = 0.5 * (propagatingFunctionStationary(- v * t[1], xmin, xmax) - propagatingFunctionStationary(v * t[1], xmin, xmax))
  # --- Update
  u[1, 1 : N]     = alpha * alpha * u[0, 2 : N + 1] / 2. + (1 - alpha * alpha) * u[0, 1 : N] + alpha * alpha * u[0, 0 : N - 1] / 2. - v * dt * 0.5 * (propagatingFunctionStationaryDerivative(x[1 : N] - xmin - v * t[0], xmin, xmax) + propagatingFunctionStationaryDerivative(x[1 : N] - xmin + v * t[0], xmin, xmax))
  # --- Right boundary condition
  u[1, N]         = 0.5 * (propagatingFunctionStationary(xmax - xmin - v * t[1], xmin, xmax) - propagatingFunctionStationary(xmax - xmin + v * t[1], xmin, xmax))

  uoldold         = u[0, 0 : N + 1]
  uold            = u[1, 0 : N + 1]
  
  for l in range(2, M + 1):

    # --- Left boundary condition
    u[l, 0]     = 0.5 * (propagatingFunctionStationary(- v * t[l], xmin, xmax) - propagatingFunctionStationary(v * t[l], xmin, xmax));   
    # --- Update
    u[l, 1 : N] = alpha * alpha * uold[2 : N + 1] + 2 * (1 - alpha * alpha) * uold[1 : N] + alpha * alpha * uold[0 : N - 1] - uoldold[1 : N] 
    # --- Right boundary condition
    u[l, N]     = 0.5 * (propagatingFunctionStationary(xmax - xmin - v * t[l], xmin, xmax) - propagatingFunctionStationary(xmax - xmin + v * t[l], xmin, xmax))
    
    uoldold     = uold
    uold        = u[l, :]

  return u

Parameters.

In [12]:
xmin  = 0                                    # --- Left boundary of the simulation domain
xmax  = 2. * np.pi                           # --- Right boundary of the simulation domain
t_0   = 0.                                   # --- Initial time
t_f   = 15.                                  # --- Final time
M     = 200                                  # --- Number of time steps
N     = 100                                  # --- Number of space mesh points
v     = 0.5                                  # --- Wave speed

Space-time discretization.

In [13]:
x, dx = np.linspace(xmin, xmax, N + 1, retstep = True)

t, dt = np.linspace(t_0, t_f, M + 1, retstep = True)
            
alpha = v * dt / dx

Define reference solution.

In [14]:
X, T = np.meshgrid(x, t)           

uFW  =  0.5 * propagatingFunctionStationary(X - xmin - v * T, xmin, xmax);
uBW  = -0.5 * propagatingFunctionStationary(X - xmin + v * T, xmin, xmax);

uRef = 0.5 * (propagatingFunctionStationary(X - xmin - v * T, xmin, xmax) - propagatingFunctionStationary(X - xmin + v * T, xmin, xmax));  

Compute solution.

In [15]:
u = leapfrogStationary(x, t, v, xmin, xmax, alpha)

Animation.

In [ ]:
fig, ax = plt.subplots()
plt.close()

ax.set_xlim(( xmin, xmax))
ax.set_ylim(( -1.1, 1.1))

line1, = ax.plot([], [], lw = 2, color = "r")
line2, = ax.plot([], [], lw = 2, color = "g")
line3, = ax.plot([], [], lw = 2, color = "c")
line4, = ax.plot([], [], lw = 2, color = "b", marker = "o")

def animate(i):
    global x
    global u, uRef, uFW, uBW
    yRef = uRef[i]
    yFW  = uFW[i]
    yBW  = uBW[i]
    y    = u[i]
    line1.set_data(x, yRef)
    line2.set_data(x, yFW)
    line3.set_data(x, yBW)
    line4.set_data(x, y)
    return (line1, line2, line3, line4)

anim = animation.FuncAnimation(fig, animate, frames = N + 1, interval = 20)
anim.save('leapfrog.mp4', fps = 30, extra_args=['-vcodec', 'libx264'])
rc('animation', html = 'jshtml') 
anim